In [1]:
##Importing necessary libraries
import math
import pandas as pd
import numpy as np
import scipy
import random
import math ##Importing Math function to use floor function.(Floor function rounds up the value to the nearest integer)
from scipy import stats
from csv import reader
random.seed(120)

In [2]:
path1 = "/mnt/c/Users/preety.tiwari/Documents/market_Risk_modelling/Datasets/historical parameters.csv"
path2 = "/mnt/c/Users/preety.tiwari/Documents/market_Risk_modelling/Datasets/rates_exchange.csv"
path3 = "/mnt/c/Users/preety.tiwari/Documents/market_Risk_modelling/Datasets/transaction_fxcash.csv"
path4 = "/mnt/c/Users/preety.tiwari/Documents/market_Risk_modelling/Datasets/rates_exchange_hp.csv"

In [3]:
def getData(path, fileext = None):
    if fileext == ".csv":
        df = pd.read_csv(path)
    elif fileext == ".xlsx" or fileext == ".xls":
        df = pd.read_excel(path)
        
    else:
        df = pd.read_csv(path)
    return df

In [4]:
##Importing dataset rates_instrument
df= getData(path1)
rates_exchange = getData(path2)

In [5]:
n = df['no_of_days']
decay_factor = df['decay_factor']
holding_period = df['holding_period']
confidence_level = df['confidence_level']
confidence_level = list(confidence_level)
confidence_level = confidence_level[0]
decay_factor = list(decay_factor)
decay_factor = decay_factor[0]

In [6]:
# Simulated Instrument rates
simulated_instruments = pd.DataFrame([])

if len(rates_exchange) != 0:
    rates_exchange["key"] = rates_exchange["currency_from"] + "_"+rates_exchange["currency_to"]

In [7]:
from copy import deepcopy
rates_ex1 = deepcopy(rates_exchange)
rates_ex1 = rates_exchange.sort_values(by = 'key').reset_index(drop = True)

In [8]:
simulated_exchange = pd.DataFrame([])

In [9]:
for i,k in rates_exchange.groupby('key'):
    j = k.filter(["currency_from", "currency_to", "rate", "key"])
    new_row = j.iloc[0].to_dict()
    print(new_row)
    new_row['simulated_rates'] = [0]
    top_row = pd.DataFrame(new_row)
    j = j.reset_index(drop = True)
    j['simulated_rates'] = np.log(j['rate']/j['rate'].shift(-1))
    j = pd.concat([top_row, j]).reset_index(drop = True)
    j["new_rate"] = j['simulated_rates'].apply(lambda x: np.exp(x)*j["rate"].iloc[0])
    simulated_exchange = simulated_exchange.append(j)

{'currency_from': 'EUR', 'currency_to': 'INR', 'rate': 82.32807398, 'key': 'EUR_INR'}
{'currency_from': 'GBP', 'currency_to': 'INR', 'rate': 95.55583361, 'key': 'GBP_INR'}
{'currency_from': 'INR', 'currency_to': 'INR', 'rate': 1.0, 'key': 'INR_INR'}
{'currency_from': 'USD', 'currency_to': 'INR', 'rate': 78.54457228, 'key': 'USD_INR'}


/tmp/ipykernel_863/3160121930.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_exchange = simulated_exchange.append(j)
/tmp/ipykernel_863/3160121930.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_exchange = simulated_exchange.append(j)
/tmp/ipykernel_863/3160121930.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_exchange = simulated_exchange.append(j)
/tmp/ipykernel_863/3160121930.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_exchange = simulated_exchange.append(j)


In [10]:
se = simulated_exchange.dropna(axis = 0)


### Transaction Fxcash

In [11]:
def get_mtm_bc(idx,tx_df, ex_df):
    
    res_df = list()
    for k, h in tx_eq.groupby('id'):
        h["keyMatch"] = h['currency'] + '_' + h['currency_base']
        keyMatch = h["keyMatch"].iloc[0]
        instrMatch = h["instrument"].iloc[0]
        unitCount = h['unit_count'].iloc[0]
        
        crss_rate_df = ex_df[ex_df['key']==keyMatch]
        cross_rate = crss_rate_df['new_rate'].iloc[0]
        
        #equity_rate_df = instr_df[instr_df['instrument'] == instrMatch]
        #equity_rate = equity_rate_df['new_value'].iloc[0]
        
        mtm_bc = cross_rate  * unitCount
        result = {'idx': idx, "env":k,"instrument": instrMatch, "unit_count":unitCount, "currency":h["currency"].iloc[0], "mtm_bc":mtm_bc}
        res_df.append(result)

    return res_df

In [12]:
tx_eq = getData(path3)

In [13]:
get_mtm_bc(251,tx_eq,se)

[{'idx': 251,
  'env': 'id1',
  'instrument': 'FXCash',
  'unit_count': 10000,
  'currency': 'USD',
  'mtm_bc': 785445.7228},
 {'idx': 251,
  'env': 'id2',
  'instrument': 'FXCash',
  'unit_count': 2000,
  'currency': 'GBP',
  'mtm_bc': 191111.66721999997},
 {'idx': 251,
  'env': 'id3',
  'instrument': 'FXCash',
  'unit_count': 5000,
  'currency': 'EUR',
  'mtm_bc': 411640.3699}]

In [14]:
final_df = pd.DataFrame()
for i in range(0, 251):
    exch_rates = simulated_exchange[simulated_exchange.index == i]
    #instr_rates = simulated_instruments[simulated_instruments.index == i]
    result = get_mtm_bc(i, tx_eq, exch_rates)
    res = pd.DataFrame(result)

    final_df = final_df.append(res)

/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_

/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_

/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_

/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_

/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_863/3146400129.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_

In [15]:
sorted_data = final_df.sort_values(by = ['env', 'idx'])

In [16]:
sorted_data = final_df.sort_values(by = ['currency','idx'])

In [17]:
##Change in mtm
def change(df):
    
        change_df = pd.DataFrame()
        #change_values = []
    
        for i,j in df.groupby('currency'):
            curr_val = j['mtm_bc'].iloc[0]
            print(curr_val)
            j['change_in_mtm'] = j['mtm_bc'].apply(lambda x: x - curr_val)
            change_df = change_df.append(j)
        
        return change_df

In [18]:
chc = change(sorted_data)

411640.3699
191111.66721999997
785445.7228


/tmp/ipykernel_863/2853993128.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)
/tmp/ipykernel_863/2853993128.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)
/tmp/ipykernel_863/2853993128.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)


In [19]:
chc

,idx,env,instrument,unit_count,currency,mtm_bc,change_in_mtm
2,0,id3,FXCash,5000,EUR,411640.369900,0.000000
2,1,id3,FXCash,5000,EUR,396593.695311,-15046.674589
2,2,id3,FXCash,5000,EUR,419882.996581,8242.626681
2,3,id3,FXCash,5000,EUR,426882.980404,15242.610504
2,4,id3,FXCash,5000,EUR,397479.810958,-14160.558942
...,...,...,...,...,...,...,...
0,246,id1,FXCash,10000,USD,791604.947081,6159.224281
0,247,id1,FXCash,10000,USD,786068.837375,623.114575
0,248,id1,FXCash,10000,USD,820574.953025,35129.230225
0,249,id1,FXCash,10000,USD,773461.674855,-11984.047945


In [20]:
sorted_data

,idx,env,instrument,unit_count,currency,mtm_bc
2,0,id3,FXCash,5000,EUR,411640.369900
2,1,id3,FXCash,5000,EUR,396593.695311
2,2,id3,FXCash,5000,EUR,419882.996581
2,3,id3,FXCash,5000,EUR,426882.980404
2,4,id3,FXCash,5000,EUR,397479.810958
...,...,...,...,...,...,...
0,246,id1,FXCash,10000,USD,791604.947081
0,247,id1,FXCash,10000,USD,786068.837375
0,248,id1,FXCash,10000,USD,820574.953025
0,249,id1,FXCash,10000,USD,773461.674855


In [21]:
chc

,idx,env,instrument,unit_count,currency,mtm_bc,change_in_mtm
2,0,id3,FXCash,5000,EUR,411640.369900,0.000000
2,1,id3,FXCash,5000,EUR,396593.695311,-15046.674589
2,2,id3,FXCash,5000,EUR,419882.996581,8242.626681
2,3,id3,FXCash,5000,EUR,426882.980404,15242.610504
2,4,id3,FXCash,5000,EUR,397479.810958,-14160.558942
...,...,...,...,...,...,...,...
0,246,id1,FXCash,10000,USD,791604.947081,6159.224281
0,247,id1,FXCash,10000,USD,786068.837375,623.114575
0,248,id1,FXCash,10000,USD,820574.953025,35129.230225
0,249,id1,FXCash,10000,USD,773461.674855,-11984.047945


In [22]:
##Total change in mtm
total_change_in_mtm = chc.groupby('idx')[['change_in_mtm']].sum()

In [23]:
total_change_in_mtm = list(total_change_in_mtm['change_in_mtm'])
total_change_in_mtm.pop(0)

0.0

In [24]:
decay_factor = 0.94

In [25]:
prob = (1 - confidence_level)
decay_weights = []
if decay_factor == 0:
    VaR_exact_fxcash_one_day = -(np.quantile(total_change_in_mtm, prob))
    total_change_in_mtm_sorted = sorted(total_change_in_mtm, reverse = True)
    nearby = np.floor(confidence_level * len(total_change_in_mtm))
    nearby = int(nearby)
    nearby_nxt = nearby + 1
    
    diff1 = pd.to_numeric(VaR_exact_fxcash + total_change_in_mtm_sorted[nearby])
    diff2 = pd.to_numeric(- total_change_in_mtm_sorted[nearby_nxt] - VaR_exact_fxcash)
    
    if (diff1 == diff2):
        VaR_nearby_fxcash_one_day = - total_change_in_mtm_sorted[nearby + 1]
    elif (diff1 < diff2):
        VaR_nearby_fxcash_one_day = -total_change_in_mtm_sorted[nearby]
    else:
        VaR_nearby_fxcash_one_day = - total_change_in_mtm_sorted[nearby + 1]
        
else:
    a = []
    a = list((chc['idx'].unique()))
    a.pop()
    for number in a:
        decay_weights.append((1-decay_factor)*(decay_factor ** number))
    
    total_mtm = chc.groupby('idx')[['mtm_bc']].sum
    a= pd.DataFrame(range(0,(len(chc['idx'].unique())-2)))
    #hp['decay_factor'].pow(a)
    #decay_weights = (1-decay_factor) * (hp['decay_factor'].pow(a))/((1 - decay_factor) ** (len(Result['env'].unique() -1)))
    total_mtm = chc.groupby('idx')['mtm_bc'].sum()
    total_mtm = list(total_mtm)
    mtm_returns = []
    
    for i in range(1,(len(chc['idx'].unique() -1))):
                   fv = total_mtm[0]
                   returns = (total_mtm[i] - fv)/fv
                   mtm_returns.append(returns)
    #decay_weights.rename(columns = {0:'decay_weights'}, inplace = True)
    decay_weights = pd.DataFrame(decay_weights, columns = ['decay_weights'])
    mtm_returns = pd.DataFrame(mtm_returns, columns = ['mtm_returns'])
    mtm = pd.concat([mtm_returns,decay_weights], axis = 1)
    mtm = mtm.sort_values(['mtm_returns']).reset_index()
    mtm['cum_decay_weights'] = np.cumsum(mtm['decay_weights'])
    alpha = 1-confidence_level
    mtm1 = mtm.loc[mtm['cum_decay_weights'] < alpha]
    
######################################################################################################
    
    if (len(mtm1['cum_decay_weights'])==0):
                  VaR_exact_fxcash_one_day = np.abs(- mtm['mtm_returns'][0] * total_mtm[0])
                  VaR_nearby_fxcash_one_day = VaR_exact_fxcash_one_day


    else:
                index = len(mtm1['cum_decay_weights'])
                                         
                interpolated_return_exact = mtm['mtm_returns'][index-1] + ((mtm['mtm_returns'][index] - mtm['mtm_returns'][index-1]) * ((1 - confidence_level) - mtm['cum_decay_weights'][index-1]) / (mtm['cum_decay_weights'][index] - mtm['cum_decay_weights'][index-1]))
    
                VaR_exact_fxcash =  abs((total_mtm[0] * (interpolated_return_exact)))
    
                if((((1 - confidence_level) - mtm['cum_decay_weights'][index-1]) < (mtm['cum_decay_weights'][index] - (1 - confidence_level) ))):
            
                            interpolated_return_nearby = mtm['mtm_returns'][index-1]
                            VAR_nearby_fxcash = abs((total_mtm[0] * (interpolated_return_nearby))) 
                             
                else:
                            index = len(mtm1['cum_decay_weights'])
                            interpolated_return_nearby = mtm['mtm_returns'][index]
                            VAR_nearby_one_day = abs((total_mtm[0] * (interpolated_return_nearby)))

In [26]:
VaR_exact_fxcash_one_day

64115.35741090239

In [27]:
VaR_nearby_fxcash_one_day

64115.35741090239

## Holding Period VAR

In [28]:
holding_period = 10
rates_exchange_hp = getData(path4)

In [30]:
# Simulated Instrument rates
simulated_instruments = pd.DataFrame([])

if len(rates_exchange_hp) != 0:
    rates_exchange_hp["key"] = rates_exchange_hp["currency_from"] + "_"+rates_exchange_hp["currency_to"]

In [31]:
simulated_exchange_hp = pd.DataFrame([])
for i,k in rates_exchange_hp.groupby('key'):
    j = k.filter(["currency_from", "currency_to", "rate", "key"])
    new_row = j.iloc[0].to_dict()
    print(new_row)
    new_row['simulated_rates'] = [0]
    top_row = pd.DataFrame(new_row)
    j = j.reset_index(drop = True)
    j['simulated_rates'] = np.log(j['rate']/j['rate'].shift(-holding_period))
    j = pd.concat([top_row, j]).reset_index(drop = True)
    j["new_rate"] = j['simulated_rates'].apply(lambda x: np.exp(x)*j["rate"].iloc[0])
    simulated_exchange_hp = simulated_exchange_hp.append(j)

{'currency_from': 'EUR', 'currency_to': 'INR', 'rate': 82.32807398, 'key': 'EUR_INR'}
{'currency_from': 'GBP', 'currency_to': 'INR', 'rate': 95.55583361, 'key': 'GBP_INR'}
{'currency_from': 'INR', 'currency_to': 'INR', 'rate': 1.0, 'key': 'INR_INR'}
{'currency_from': 'USD', 'currency_to': 'INR', 'rate': 78.54457228, 'key': 'USD_INR'}


/tmp/ipykernel_863/3282333018.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_exchange_hp = simulated_exchange_hp.append(j)
/tmp/ipykernel_863/3282333018.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_exchange_hp = simulated_exchange_hp.append(j)
/tmp/ipykernel_863/3282333018.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_exchange_hp = simulated_exchange_hp.append(j)
/tmp/ipykernel_863/3282333018.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_exchange_hp = simulated_exchange_hp.append(j)


In [34]:
simulated_exchange_hp = simulated_exchange_hp.dropna(axis = 0)

In [37]:
def get_mtm_bc(idx,tx_df, ex_df):
    
    res_df = list()
    for k, h in tx_eq.groupby('id'):
        h["keyMatch"] = h['currency'] + '_' + h['currency_base']
        keyMatch = h["keyMatch"].iloc[0]
        instrMatch = h["instrument"].iloc[0]
        unitCount = h['unit_count'].iloc[0]
        
        crss_rate_df = ex_df[ex_df['key']==keyMatch]
        cross_rate = crss_rate_df['new_rate'].iloc[0]
        
        #equity_rate_df = instr_df[instr_df['instrument'] == instrMatch]
        #equity_rate = equity_rate_df['new_value'].iloc[0]
        
        mtm_bc = cross_rate  * unitCount
        result = {'idx': idx, "env":k,"instrument": instrMatch, "unit_count":unitCount, "currency":h["currency"].iloc[0], "mtm_bc":mtm_bc}
        res_df.append(result)

    return res_df

In [38]:
tx_eq = getData(path3)

In [39]:
get_mtm_bc(251,tx_eq,simulated_exchange_hp)

[{'idx': 251,
  'env': 'id1',
  'instrument': 'FXCash',
  'unit_count': 10000,
  'currency': 'USD',
  'mtm_bc': 785445.7228},
 {'idx': 251,
  'env': 'id2',
  'instrument': 'FXCash',
  'unit_count': 2000,
  'currency': 'GBP',
  'mtm_bc': 191111.66721999997},
 {'idx': 251,
  'env': 'id3',
  'instrument': 'FXCash',
  'unit_count': 5000,
  'currency': 'EUR',
  'mtm_bc': 411640.3699}]

In [40]:
result = pd.DataFrame()
for i in range(0, 251):
    exch_rates = simulated_exchange_hp[simulated_exchange_hp.index == i]
    #instr_rates = simulated_instruments[simulated_instruments.index == i]
    ress = get_mtm_bc(i, tx_eq, exch_rates)
    res = pd.DataFrame(ress)

    result = result.append(res)

/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/

/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/

/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/

/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/ipykernel_863/1460187635.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
/tmp/

In [42]:
sorted_data = result.sort_values(by = ['env', 'idx'])

In [44]:
def change(df):
    
        change_df = pd.DataFrame()
        #change_values = []
    
        for i,j in df.groupby('currency'):
            curr_val = j['mtm_bc'].iloc[0]
            print(curr_val)
            j['change_in_mtm'] = j['mtm_bc'].apply(lambda x: x - curr_val)
            change_df = change_df.append(j)
        
        return change_df

In [45]:
chc = change(sorted_data)

411640.3699
191111.66721999997
785445.7228


/tmp/ipykernel_863/1879246163.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)
/tmp/ipykernel_863/1879246163.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)
/tmp/ipykernel_863/1879246163.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)


In [47]:
total_change_in_mtm = chc.groupby('idx')[['change_in_mtm']].sum()
total_change_in_mtm = list(total_change_in_mtm['change_in_mtm'])
total_change_in_mtm.pop(0)

0.0

In [49]:
prob = (1 - confidence_level)
decay_weights = []
if decay_factor == 0:
    VaR_exact_fxcash_hp = -(np.quantile(total_change_in_mtm, prob))
    total_change_in_mtm_sorted = sorted(total_change_in_mtm, reverse = True)
    nearby = np.floor(confidence_level * len(total_change_in_mtm))
    nearby = int(nearby)
    nearby_nxt = nearby + 1
    
    diff1 = pd.to_numeric(VaR_exact_fxcash_hp + total_change_in_mtm_sorted[nearby])
    diff2 = pd.to_numeric(- total_change_in_mtm_sorted[nearby_nxt] - VaR_exact_fxcash_hp)
    
    if (diff1 == diff2):
        VaR_nearby_fxcash_hp = - total_change_in_mtm_sorted[nearby + 1]
    elif (diff1 < diff2):
        VaR_nearby_fxcash_hp = -total_change_in_mtm_sorted[nearby]
    else:
        VaR_nearby_fxcash_hp = - total_change_in_mtm_sorted[nearby + 1]
        
else:
    a = []
    a = list((chc['idx'].unique()))
    a.pop()
    for number in a:
        decay_weights.append((1-decay_factor)*(decay_factor ** number))
    
    total_mtm = chc.groupby('idx')[['mtm_bc']].sum
    a= pd.DataFrame(range(0,(len(chc['idx'].unique())-2)))
    #hp['decay_factor'].pow(a)
    #decay_weights = (1-decay_factor) * (hp['decay_factor'].pow(a))/((1 - decay_factor) ** (len(Result['env'].unique() -1)))
    total_mtm = chc.groupby('idx')['mtm_bc'].sum()
    total_mtm = list(total_mtm)
    mtm_returns = []
    
    for i in range(1,(len(chc['idx'].unique() -1))):
                   fv = total_mtm[0]
                   returns = (total_mtm[i] - fv)/fv
                   mtm_returns.append(returns)
    #decay_weights.rename(columns = {0:'decay_weights'}, inplace = True)
    decay_weights = pd.DataFrame(decay_weights, columns = ['decay_weights'])
    mtm_returns = pd.DataFrame(mtm_returns, columns = ['mtm_returns'])
    mtm = pd.concat([mtm_returns,decay_weights], axis = 1)
    mtm = mtm.sort_values(['mtm_returns']).reset_index()
    mtm['cum_decay_weights'] = np.cumsum(mtm['decay_weights'])
    alpha = 1-confidence_level
    mtm1 = mtm.loc[mtm['cum_decay_weights'] < alpha]
    
######################################################################################################
    
    if (len(mtm1['cum_decay_weights'])==0):
                  VaR_exact_fxcash_hp = np.abs(- mtm['mtm_returns'][0] * total_mtm[0])
                  VaR_nearby_fxcash_hp = VaR_exact_fxcash_one_day


    else:
                index = len(mtm1['cum_decay_weights'])
                                         
                interpolated_return_exact = mtm['mtm_returns'][index-1] + ((mtm['mtm_returns'][index] - mtm['mtm_returns'][index-1]) * ((1 - confidence_level) - mtm['cum_decay_weights'][index-1]) / (mtm['cum_decay_weights'][index] - mtm['cum_decay_weights'][index-1]))
    
                VaR_exact_fxcash_hp =  abs((total_mtm[0] * (interpolated_return_exact)))
    
                if((((1 - confidence_level) - mtm['cum_decay_weights'][index-1]) < (mtm['cum_decay_weights'][index] - (1 - confidence_level) ))):
            
                            interpolated_return_nearby = mtm['mtm_returns'][index-1]
                            VaR_nearby_fxcash_hp = abs((total_mtm[0] * (interpolated_return_nearby))) 
                             
                else:
                            index = len(mtm1['cum_decay_weights'])
                            interpolated_return_nearby = mtm['mtm_returns'][index]
                            VaR_nearby_fxcash_hp = abs((total_mtm[0] * (interpolated_return_nearby)))

In [50]:
VaR_exact_fxcash_hp

45079.64370729235

In [51]:
VaR_nearby_fxcash_hp

45711.770731156226

In [52]:
VaR_exact_fxcash_one_day

64115.35741090239

In [53]:
VaR_nearby_fxcash_one_day

64115.35741090239

In [ ]:
VaR_hp = pd.DataFrame()